# Set Up/Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# conda activate "/pscratch/sd/m/mansisak/memorization/env/"

In [3]:
# !pip list

In [4]:
#pip install accelerate

In [5]:
import torch
import os
from transformers import AutoModelForCausalLM
from torch.utils.data import DataLoader
from tqdm import tqdm

### TODO: figure out automatically
# device = "cpu"
device = "cuda"

/global/cfs/cdirs/m636/geniesse/envs/memorization/lib/python3.10/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


# Load in Models

In [6]:
def print_edited_model_paths(parent_path):
    
    total_exp = 0
    for model_name in ['pythia-6.9b-deduped', 'pythia-2.8b-deduped']:
        y_idx = 0
        for step in [36000, 72000, 108000, 143000]:
            for loc_method in [
                  "act",
                  "hc",
                  "slim",
                  "durable",
                  "durable_agg",
                  "random",
                  "random_greedy",
                  "greedy",
              ]:
                
                for ratio in [0.00001, 0.0001, 0.001, 0.01, 0.05, 0.1, 0.25, 0.3]:
                    result_path = f"{parent_path}{step}/EleutherAI_edit/{loc_method}/mem/{ratio}"
                    if loc_method not in ["random", "random_greedy"]:
                        if ratio >= 0.1:
                            continue

                    # this ratio is too small for neuron-level methods
                    if loc_method in ["zero", "hc", "ig", "slim", "act"]:
                        if ratio <= 0.0001:
                            continue

                    if loc_method in ["greedy"]:
                        if ratio > 0.00001:
                            continue
                            
                    ######
                    if loc_method in ["greedy", "durable", "durable_agg","act"]:
                        model_path = f'{result_path}/{model_name}'
                        total_exp += 1

                    if loc_method in ["slim", "hc"]:
                        for epochs in [1, 10, 20]:
                            total_exp += 1
                            model_path = f'{result_path}/{epochs}/1000/0.1/0.1/{model_name}'                      

                    if loc_method in ["random", "random_greedy"]:
                        for epochs in [1, 10, 20]:
                            total_exp += 1
                            model_path = f'{result_path}/{epochs}/0.1/0.9/0.0005/{model_name}'
                    if os.path.isfile(model_path):
                        print("edited model exists:", model_path)
                    else:
                        print("edited model doesn't exist yet: ", model_path)

    print("total_expeirments: ", total_exp)



In [7]:
# parent_path = "/pscratch/sd/m/mansisak/memorization/model_ckpts/"
parent_path = "/pscratch/sd/m/mansisak/memorization/model_ckpts/"

print_edited_model_paths(parent_path)

edited model exists: /pscratch/sd/m/mansisak/memorization/model_ckpts/36000/EleutherAI_edit/act/mem/0.001/pythia-6.9b-deduped
edited model exists: /pscratch/sd/m/mansisak/memorization/model_ckpts/36000/EleutherAI_edit/act/mem/0.01/pythia-6.9b-deduped
edited model exists: /pscratch/sd/m/mansisak/memorization/model_ckpts/36000/EleutherAI_edit/act/mem/0.05/pythia-6.9b-deduped
edited model exists: /pscratch/sd/m/mansisak/memorization/model_ckpts/36000/EleutherAI_edit/hc/mem/0.001/20/1000/0.1/0.1/pythia-6.9b-deduped
edited model exists: /pscratch/sd/m/mansisak/memorization/model_ckpts/36000/EleutherAI_edit/hc/mem/0.01/20/1000/0.1/0.1/pythia-6.9b-deduped
edited model exists: /pscratch/sd/m/mansisak/memorization/model_ckpts/36000/EleutherAI_edit/hc/mem/0.05/20/1000/0.1/0.1/pythia-6.9b-deduped
edited model exists: /pscratch/sd/m/mansisak/memorization/model_ckpts/36000/EleutherAI_edit/slim/mem/0.001/20/1000/0.1/0.1/pythia-6.9b-deduped
edited model exists: /pscratch/sd/m/mansisak/memorization/mo

In [8]:
# Example of how to load in a model:

model = AutoModelForCausalLM.from_pretrained(
    "EleutherAI/pythia-2.8b-deduped",
    torch_dtype=torch.float16,
    device_map=device
)

# sd_path = '/pscratch/sd/m/mansisak/memorization/model_ckpts/old_run/143000/EleutherAI_edit/random_greedy/mem/0.001/20/0.1/0.9/0.0005/pythia-2.8b-deduped'
sd_paths = dict(
    unedited = None,
    good_edit = "/pscratch/sd/m/mansisak/memorization/model_ckpts/143000/EleutherAI_edit/random_greedy/mem/0.001/20/0.1/0.9/0.0005/pythia-2.8b-deduped",
    bad_edit = "/pscratch/sd/m/mansisak/memorization/model_ckpts/143000/EleutherAI_edit/random/mem/0.1/20/0.1/0.9/0.0005/pythia-2.8b-deduped",
)

# choose which model to load
use_model = "bad_edit"
sd_path = sd_paths.get(use_model)

if sd_path is not None:
    sd = torch.load(sd_path, map_location=device)["model_state_dict"]
    if "random" in sd_path:
        for k in sd:
            if "4h" in k:
                #rint(sd[k].shape)
                #rint(k)
                sd[k] = sd[k].T
    model.load_state_dict(sd, assign=True)
model.eval()

/tmp/ipykernel_1966595/3153447877.py:27: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3637.)
  sd[k] = sd[k].T


GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50304, 2560)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-31): 32 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear(in_features=2560, out_features=7680, bias=True)
          (dense): Linear(in_features=2560, out_features=2560, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=2560, out_features=10240, bias=True)
          (dense_4h_to_h): Linear(in_features=10240, out_features=2560, bias=True)


In [9]:
!ls -ls /pscratch/sd/m/mansisak/memorization/model_ckpts/143000

total 4
4 drwxr-xr-x 11 mansisak mansisak 4096 Sep  3 06:26 EleutherAI_edit


In [10]:
model.device

device(type='cuda', index=0)

# Load in Data

In [11]:
random_data = torch.load("/pscratch/sd/m/mansisak/memorization/src/data/pythia_mem_data/pile_random_batch.pt")
random_data_pile = torch.reshape(random_data[0:2040], (3264, 80)).to(device)

random_data = random_data_pile[0:1632]
random_subset_data = random_data_pile[0:10] # for testing purposes
extra_data = random_data_pile[1632:]
random_dataloader = DataLoader(random_data, batch_size=1, shuffle=False)
random_subset_dataloader = DataLoader(random_subset_data, batch_size=1, shuffle=False)
random_dataloader

In [12]:
# need (inputs, targets) format for PyHessian
hessian_dataloader = DataLoader([*zip(random_data, random_data)], batch_size=1, shuffle=False)
hessian_dataloader

In [13]:
# need (inputs, targets) format for PyHessian
hessian_subset_dataloader = DataLoader([*zip(random_subset_data, random_subset_data)], batch_size=1, shuffle=False)
hessian_subset_dataloader

# Model Inference

In [14]:
def perplexity(model, dataloader):
    avg_metric = 0
    for i, batch in enumerate(dataloader):
        with torch.no_grad():
            model_output = model(batch, labels=batch)
        loss = model_output.loss
        loss_exp = torch.exp(loss)
        if torch.isinf(loss_exp):
            print(f"Skipping invalid loss...")
            print(f"    {i=}, loss={loss.item()}, exp(loss)={torch.exp(loss).item()}")
        avg_metric += loss_exp
        # print(loss.item())
    return avg_metric / len(dataloader)


def perplexity_criterion(model_output, targets=None):
    loss = model_output.loss
    return torch.exp(loss)


def average_loss(model, dataloader):
    avg_metric = 0
    for i, batch in enumerate(dataloader):
        with torch.no_grad():
            model_output = model(batch, labels=batch)
        loss = model_output.loss
        avg_metric += loss
        # print(loss.item())
    return avg_metric / len(dataloader)


from torch.nn import CrossEntropyLoss
def loss_criterion(model_output, inputs):
    # Shift so that tokens < n predict n
    shift_labels = inputs[..., 1:].contiguous()
    shift_logits = model_output.logits[..., :-1, :].contiguous()
    # Calculate per-token loss
    loss_fct = CrossEntropyLoss(reduction="none")
    loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))
    # Resize and average loss per sample
    loss_per_sample = loss.view(shift_logits.size(0), shift_logits.size(1)).mean(axis=1)
    return (loss_per_sample).mean()


In [ ]:
average_loss(model, random_dataloader)

In [ ]:
loss_criterion(model(random_data[:10]), random_data[:10])

# Loss Landscape stuff

In [15]:
import numpy as np
from pyhessian import hessian
from pyhessian.utils import normalization
import copy

import matplotlib.pyplot as plt

In [16]:
def get_params(model_orig, model_perb, direction, alpha):
    for m_orig, m_perb, d in zip(
        model_orig.parameters(), model_perb.parameters(), direction
    ):
        m_perb.data = m_orig.data + alpha * d
    return model_perb

## 2D random directions (loss-landscapes)

In [ ]:
# libraries
import copy
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import numpy as np
import torch
import torch.nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.datasets as datasets
from tqdm import tqdm

matplotlib.rcParams['figure.figsize'] = [18, 12]

import loss_landscapes
import loss_landscapes.metrics

In [ ]:
# data that the evaluator will use when evaluating loss
# x = iter(random_subset_dataloader).__next__()
metric = loss_landscapes.metrics.Loss(loss_criterion, random_subset_data, random_subset_data)
metric(model)

In [ ]:
# data that the evaluator will use when evaluating loss
x = iter(random_subset_dataloader).__next__()
metric = loss_landscapes.metrics.Loss(loss_criterion, x.to(device), x.to(device))
metric(model.to(device))

In [ ]:
device = "cuda"

with torch.no_grad():
    model.eval()
    model_perb = copy.deepcopy(model)
    model_perb.eval()
    
    model = model.to(device)
    model_perb = model_perb.to(device)
    

In [ ]:
STEPS = 5 
DIST = 1

loss_data_fin = loss_landscapes.random_plane(
    model_perb, metric, 
    DIST, STEPS, 
    normalization=None, #'layer', 
    deepcopy_model=False,
    device=device,
)


In [ ]:
from loss_landscapes.model_interface.model_wrapper import ModelWrapper, wrap_model
from loss_landscapes.model_interface.model_parameters import rand_u_like, orthogonal_to
from loss_landscapes.metrics.metric import Metric

In [ ]:

distance = 100
steps = 5

model_start_wrapper = wrap_model(model_perb)

start_point = model_start_wrapper.get_module_parameters()
dir_one = rand_u_like(start_point)
dir_two = rand_u_like(start_point)
# dir_two = orthogonal_to(dir_one)

# TODO: add to() method to ModelParameters
# start_point = start_point.to(device)
dir_one.parameters = [p.to(device) for p in dir_one.parameters]
dir_two.parameters = [p.to(device) for p in dir_two.parameters]



In [ ]:

# # scale to match steps and total distance
# dir_one.mul_(((start_point.model_norm() * distance) / steps) / dir_one.model_norm())
# dir_two.mul_(((start_point.model_norm() * distance) / steps) / dir_two.model_norm())
# # Move start point so that original start params will be in the center of the plot
# dir_one.mul_(steps / 2)
# dir_two.mul_(steps / 2)
# start_point.sub_(dir_one)
# start_point.sub_(dir_two)
# dir_one.truediv_(steps / 2)
# dir_two.truediv_(steps / 2)


In [ ]:
dir_one.parameters

In [ ]:
# start_point.parameters

In [ ]:

data_matrix = []
# evaluate loss in grid of (steps * steps) points, where each column signifies one step
# along dir_one and each row signifies one step along dir_two. The implementation is again
# a little convoluted to avoid constructive operations. Fundamentally we generate the matrix
# [[start_point + (dir_one * i) + (dir_two * j) for j in range(steps)] for i in range(steps].
# for i in tqdm.tqdm(range(steps)):
for i in range(steps):
    data_column = []

    for j in range(steps):
        # for every other column, reverse the order in which the column is generated
        # so you can easily use in-place operations to move along dir_two
        if i % 2 == 0:
            start_point.add_(dir_two)
            loss = metric(model_start_wrapper)
            data_column.append(loss)
            print(f"({i=}, {j=}) {loss=}")
        else:
            start_point.sub_(dir_two)
            loss = metric(model_start_wrapper)
            data_column.insert(0, loss)
            print(f"({i=}, {j=}) {loss=}")


    data_matrix.append(data_column)
    start_point.add_(dir_one)

data_matrix = np.array(data_matrix)

In [ ]:
data_matrix

## plotting

In [ ]:
plt.contour(loss_data_fin, levels=50)
plt.title(f'Loss Contours around Trained Model ({use_model})')
plt.show()

In [ ]:
fig = plt.figure()
ax = plt.axes(projection='3d')
X = np.array([[j for j in range(STEPS)] for i in range(STEPS)])
Y = np.array([[i for _ in range(STEPS)] for i in range(STEPS)])
ax.plot_surface(X, Y, loss_data_fin, rstride=1, cstride=1, cmap='viridis', edgecolor='none')
ax.set_title('Surface Plot of Loss Landscape')
fig.show()

In [ ]:
# for creating a responsive plot 
# %matplotlib widget

# importing required libraries 
from mpl_toolkits.mplot3d import axes3d
import matplotlib.pyplot as plt 

fig = plt.figure()

# Reshape the data_matrix into a 41x41 grid for plotting
Z = data_matrix.reshape((STEPS, STEPS))

# Generate X and Y coordinates (41x41 grid)
x = np.linspace(0, STEPS-1, STEPS)
y = np.linspace(0, STEPS-1, STEPS)
X, Y = np.meshgrid(x, y)

ax = fig.add_subplot(111, projection='3d')
# Plot the 3D surface with filled colors
surf = ax.plot_surface(X, Y, Z, cmap='viridis', edgecolor='none')

# Add a color bar for reference
fig.colorbar(surf, ax=ax, shrink=0.5, aspect=5)

# Set labels
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

plt.show()

## 2D random directions

In [17]:
### TODO: why does normalization make the direction all zeros ...

# set random seed
seed = 123
np.random.seed(seed)
torch.manual_seed(seed)

# generate random vector to do the loss plot

v = [torch.randn_like(p) for p in model.parameters()]
# v = normalization(v)

u = [torch.randn_like(p) for p in model.parameters()]
# u = normalization(u)
from pyhessian import orthnormal, group_add, group_product
# u = orthnormal(u, [v])
# u = group_add(u, v, alpha=-group_product(u, v))

# TODO: check to make sure orthoganal
random_directions = [u, v]

In [18]:
u

[tensor([[ 0.0685, -0.7305,  0.3538,  ..., -0.5542,  2.0664, -0.5356],
         [ 0.7666, -1.5811,  0.1115,  ..., -0.3232,  1.0518, -1.2393],
         [ 0.3999,  0.5181,  0.2529,  ..., -1.1670,  1.9053,  0.0703],
         ...,
         [ 1.2363, -1.5322, -0.7075,  ...,  1.4551, -1.5234,  0.4578],
         [-0.6777,  0.5127,  0.2998,  ...,  0.9204,  1.5596,  0.3123],
         [-0.4146,  1.7275,  1.5410,  ..., -0.8726, -1.8770,  0.8667]],
        device='cuda:0', dtype=torch.float16),
 tensor([ 1.0234, -0.1279, -0.0321,  ...,  1.3887,  1.1758,  0.4468],
        device='cuda:0', dtype=torch.float16),
 tensor([-0.8765,  1.3926,  1.3926,  ..., -2.2578, -0.2913,  1.4004],
        device='cuda:0', dtype=torch.float16),
 tensor([ 0.9829, -0.2517,  1.3848,  ...,  1.7217,  0.8096,  0.3628],
        device='cuda:0', dtype=torch.float16),
 tensor([ 0.5396, -2.1289,  2.2480,  ..., -0.6475, -0.3787,  0.6860],
        device='cuda:0', dtype=torch.float16),
 tensor([[ 0.0514, -0.1802, -0.5098,  ...,  

In [22]:
###############################################################################
# Calculate the Random landscape
###############################################################################
# STEPS = 41
STEPS = 3 # for testing
POINTS = STEPS * STEPS

DISTANCE = 100 # 0.0005
DISTANCE = 1 # for testing
START = -1 * DISTANCE
END = 1 * DISTANCE

DIM = 2

lams = np.linspace(START, END, STEPS).astype(np.float32)

with torch.no_grad():
    model.eval()
    model_perb = copy.deepcopy(model)
    model_perb.eval()

# Generate the loss values array using BFS
# Create a coordinate array for loss values
loss_coordinates_list = []
for i in range(STEPS):
    for j in range(STEPS):
        loss_coordinates_list.append((i, j))
loss_coordinates = np.array(loss_coordinates_list)
print(loss_coordinates.shape)

# Create a data matrix to store loss values
data_matrix = np.empty([POINTS, 1], dtype=float)
data_matrix.fill(-1)
print(data_matrix.shape)

# x, y = iter(train_loader).__next__()

# calculate the hessian loss values
for j in tqdm(
    range(POINTS), desc="Calculating sampling loss values in the subspace"
):
    # adjust the model and fill with a loss with corresponding model parameters
    next_pos = tuple(loss_coordinates[j])
    with torch.no_grad():
        model_current = copy.deepcopy(model)
    for i in range(DIM):
        model_perb = get_params(
            model_current, model_perb, random_directions[i], lams[next_pos[i]]
        )
        with torch.no_grad():
            model_current = copy.deepcopy(model_perb)
    # calculate the loss value
    with torch.no_grad():
        # data_matrix[j] = criterion(model_current(x), y).item()
        data_matrix[j] = average_loss(model_current, random_subset_dataloader).item()
        # print("Loss value: ", data_matrix[j])

print(data_matrix)

(9, 2)
(9, 1)


Calculating sampling loss values in the subspace: 100%|██████████| 9/9 [00:06<00:00,  1.47it/s]

[[     nan]
 [     nan]
 [     nan]
 [     nan]
 [105.1875]
 [     nan]
 [     nan]
 [     nan]
 [     nan]]


In [ ]:
# for creating a responsive plot 
# %matplotlib widget

# importing required libraries 
from mpl_toolkits.mplot3d import axes3d
import matplotlib.pyplot as plt 

fig = plt.figure()

# Reshape the data_matrix into a 41x41 grid for plotting
Z = data_matrix.reshape((STEPS, STEPS))

# Generate X and Y coordinates (41x41 grid)
x = np.linspace(0, STEPS-1, STEPS)
y = np.linspace(0, STEPS-1, STEPS)
X, Y = np.meshgrid(x, y)

ax = fig.add_subplot(111, projection='3d')
# Plot the 3D surface with filled colors
surf = ax.plot_surface(X, Y, Z, cmap='viridis', edgecolor='none')

# Add a color bar for reference
fig.colorbar(surf, ax=ax, shrink=0.5, aspect=5)

# Set labels
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

plt.show()